In [ ]:
import torch
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy.linalg import inv
from numpy.random import multivariate_normal
from notebook.services.config import ConfigManager
from traitlets.config.manager import BaseJSONConfigManager
import matplotlib as mpl
import math
from scipy.integrate import odeint, solve_ivp
import random
import torch.nn as nn



# set jupyter configurations
%matplotlib inline
%config InlineBackend.figure_format='retina'
default_dpi = mpl.rcParamsDefault['figure.dpi']
mpl.rcParams['figure.dpi'] = default_dpi*1.2


# set global variables
gt_color = 'r'
data_color = '#0000FF'
linreg_color = '#519872'
abc_color = 'purple'
sbi_color  = '#DE1A1A'

def plotting_boilerplate(ax, xlabel='',ylabel='',title='',xlim=None,ylim=None,legend=True, grid=False):
    """ Helper function to avoid wasting cell space on plotting code. Feel free to extend. """
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if legend:
        ax.legend()
    if grid:
        ax.grid(True)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_title(title)
    plt.show()

### 1.4.1 Ballistic motion problem

In [ ]:
def projectile_sim(launch_velocity, launch_angle, drag_coefficient, sample=True, plot=True):
#     c = 0.47
    """ Blackbox simulator"""
    deriv = lambda t,u: (u[1],
                         -.5 * drag_coefficient * 1.28 * 0.008/0.2 * np.hypot(u[1],u[3]) * u[1],
                         u[3],
                         -.5 * drag_coefficient * 1.28 * 0.008/0.2 * np.hypot(u[1], u[3]) * u[3] - 9.81)
    
    # Initial conditions: x0, v0_x, z0, v0_z.
    u0 = 0, launch_velocity * np.cos(np.radians(launch_angle)), 0., launch_velocity * np.sin(np.radians(launch_angle))
    # Integrate up to tf unless we hit the target sooner.
    t0, tf = 0, 50

    solution = solve_ivp(deriv, (t0, tf), u0, dense_output=True)#,
    # A fine grid of time points from 0 until impact time.
    t = np.linspace(0, 10, 100)
    # Retrieve the solution for the time grid and plot the trajectory.
    solution = solution.sol(t)
    x, z = solution[0], solution[2]
    
    samples = z + np.random.randn(x.shape[0]) * 3

    if plot:
        fig,ax = plt.subplots()
        ax.plot(x, z,label='Ground truth', color=gt_color)
        if sample:
            ax.scatter(x, samples, label='Samples', marker='x', color=data_color)
        plotting_boilerplate(ax, xlim=(0,250), ylim=0, xlabel='x /m', ylabel='x /m', legend=True)

    return dict(param=(launch_velocity, launch_angle), x=x, z=z, samples=samples)

ballistic_motion = interactive(projectile_sim, launch_velocity=(50,250), launch_angle=(10,45), drag_coefficient=(0.4,0.7,0.01), sample=False)
ballistic_motion 


### 1.4.2 Linear regression as baseline

In [ ]:
# get data from ground truth
ballistic_param = np.array(ballistic_motion.result['param'])
data = ballistic_motion.result['samples']
X = ballistic_motion.result['x']

# # linear regression baseline
features = np.array([X**0, X**1, X**2]).T
linreg_param = inv(features.T @ features).dot(features.T @ data)
linreg_reconstruction = features @ linreg_param

In [ ]:
# plotting
fig, ax = plt.subplots()
ax.scatter(X, data, marker='x', label='Samples', color=data_color, alpha=0.3)
ax.plot(X, linreg_reconstruction, 'k', label = 'Linear regression', color=linreg_color)
plotting_boilerplate(ax, legend=True, ylim=0, xlabel='$x$', ylabel='$f_x$')

### 1.4.3 Try to reproduce data manually 🛠️

In [ ]:
def distance(distance_function, prediction, data):
    """ Second-order function that takes in a distance function and its arguments and returns the result. """
    return distance_function(prediction, data)



# sample parameters from a uniform distribution
param = np.array([np.random.uniform(200, 350), np.random.uniform(40, 50)])


# pass parameters to simulator and see result
simulation = projectile_sim(300, 30, 0.47, plot=False)
Y = simulation['z']
param=simulation['param']


# plotting
fig, ax = plt.subplots()
ax.plot(X,Y, label='Reconstruction', color=abc_color)
ax.scatter(X, data, marker='x', label='Samples', color=data_color, alpha=0.3)


# write own distance functions
mse = lambda prediction, data: np.square(prediction - data).mean()
chebyshev = lambda prediction, data: max([np.abs(data[i]-prediction[i]) for i in range(data.shape[0])])

# assesed visually, what would be a good acceptance threshold?
plotting_boilerplate(ax, ylim=0, legend=True,
                     title=f'distance for $v_0={param[0]:.1f}$ and $\\theta={param[1]:.1f}$: {distance(chebyshev, Y, data):.2f}')

### 1.4.4 Automate the search for parameters 🛠️

In [ ]:
num_simulations = 1000
# best_param = None
accepted_params=[]
for _ in range(num_simulations):
    lauch_velocity, launch_angle, drag_coefficient = np.array([np.random.uniform(50, 500), np.random.uniform(30, 60), np.random.uniform(0.3, 0.5)])
    # pass to simulator
    prediction = projectile_sim(lauch_velocity, launch_angle, drag_coefficient, plot=False)
    x, y = prediction['x'], prediction['z']
    # compute distance
    dist = distance(chebyshev, y, data)
    # check if within epsilon
    if dist < 15:
        # if so return parameters
        accepted_params.append((lauch_velocity, launch_angle, drag_coefficient))
print(len(accepted_params))

In [ ]:
# plot with found parameters
launch_velocity, launch_angle, drag_coefficient = random.choice(accepted_params)
sim_result = projectile_sim(launch_velocity, launch_angle, drag_coefficient, plot=False)

# plotting
fig, ax = plt.subplots()
ax.plot(sim_result['x'], sim_result['z'], label='Reconstruction', color=abc_color)
ax.scatter(X, data, marker='x', label='Samples', color=data_color, alpha=0.3)
plotting_boilerplate(ax, ylim=0, legend=True)


- You just discovered rejection ABC!
- Reflect on how choice of $\epsilon$, $d$ was made

### 1.4.5 When does this break?

In [ ]:
def d0_dt(theta, t,b,m,g,l):
    theta1 = theta[0]
    theta2 = theta[1]
    dtheta1_dt = theta2
    dtheta2_dt = -(b/m * theta2) - (g/l * math.sin(theta1))
    dtheta_dt = [ dtheta1_dt, dtheta2_dt]
    
    return dtheta_dt


def pendulum(b,m,l, sample=True):
    theta_0 = [0,3]
    t = np.linspace(0,20,150)    
    # solve ODE
    solution = odeint(d0_dt, theta_0,t,args =(b,m,9.81,l))
    angular_velocity_measurements =  solution[:,0] + np.random.randn(t.shape[0]) * 0.2

    
    # plotting
    fig, ax = plt.subplots()
    ax.plot(t, solution[:,0], color = gt_color, linestyle = '-',linewidth = 2,label =r'$ \frac{d\theta_1}{dt}$ = $\theta_2$')
    if sample: # and measurements
        ax.scatter(t,angular_velocity_measurements,marker='x', color=data_color, label='measurements')
    plotting_boilerplate(ax, xlabel='$t$', grid=True, ylabel='Angular displacement $\\theta$', legend=True)   
    
    return dict(param=(b,m,l), solution=solution, data=angular_velocity_measurements)


damped_pendulum = interactive(pendulum, b=(0,1,0.1), m=(0.01,5,0.1), l=(1,4,0.5), sample=True)
damped_pendulum

### 1.4.6 What can we do about this?
- What summary statistics for this data would come to your  mind?
- Can we learn summary statistics with neural networks?

### 1.4.7 Introduce SBI as a potentially useful approach to solve our problem

In [ ]:
num_dim = 3

# define an initial prior
prior = utils.BoxUniform(low=torch.Tensor([0,0.01,1]), high=torch.Tensor([1,5,4]))

def pendulum_sim(param, sample=True):
    b,m,l = param
    theta_0 = [0,3]
    t = np.linspace(0,20,150)    
    # solve ODE
    solution = odeint(d0_dt, theta_0,t,args =(b,m,9.81,l))
    angular_velocity_measurements =  solution[:,0] + np.random.randn(t.shape[0]) * 0.2
    return angular_velocity_measurements


posterior = infer(pendulum_sim, prior, method='SNPE', num_simulations=500)

In [ ]:
measurement = damped_pendulum.result['data']
samples = posterior.sample((10000,), x=measurement)
log_probability = posterior.log_prob(samples, x=measurement)
_ = analysis.pairplot(samples, figsize=(6,6))

In [ ]:
_, ax = plt.subplots()
ax.plot(pendulum_sim(samples.mean(dim=0).numpy()), label='sbi reconstruction', color=sbi_color)
ax.plot(measurement, label='measurement', color=data_color)
plotting_boilerplate(ax, legend=True, xlabel='$t$', grid=True, ylabel='Angular displacement $\\theta$')